In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/26 13:16:13 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/26 13:16:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e2b603d6-7de8-4594-bb2e-dcbbc9e058ab;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 1339ms :: artifacts dl 48ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	-----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/erp/PX_CAT_G1V2.csv", header=True, inferSchema=True)
df.show()

25/05/26 13:16:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+-----+-----------+-----------------+-----------+
|   ID|        CAT|           SUBCAT|MAINTENANCE|
+-----+-----------+-----------------+-----------+
|AC_BR|Accessories|       Bike Racks|        Yes|
|AC_BS|Accessories|      Bike Stands|         No|
|AC_BC|Accessories|Bottles and Cages|         No|
|AC_CL|Accessories|         Cleaners|        Yes|
|AC_FE|Accessories|          Fenders|         No|
|AC_HE|Accessories|          Helmets|        Yes|
|AC_HP|Accessories|  Hydration Packs|         No|
|AC_LI|Accessories|           Lights|        Yes|
|AC_LO|Accessories|            Locks|        Yes|
|AC_PA|Accessories|         Panniers|         No|
|AC_PU|Accessories|            Pumps|        Yes|
|AC_TT|Accessories|  Tires and Tubes|        Yes|
|BI_MB|      Bikes|   Mountain Bikes|        Yes|
|BI_RB|      Bikes|       Road Bikes|        Yes|
|BI_TB|      Bikes|    Touring Bikes|        Yes|
|CL_BS|   Clothing|       Bib-Shorts|         No|
|CL_CA|   Clothing|             Caps|         No|


In [3]:
from pyspark.sql.functions import expr

#transformasi
df_transformed = df.select (
    expr("id"),
    expr("cat"),
    expr("subcat"),
    expr("maintenance")
)

df_transformed.show()

+-----+-----------+-----------------+-----------+
|   id|        cat|           subcat|maintenance|
+-----+-----------+-----------------+-----------+
|AC_BR|Accessories|       Bike Racks|        Yes|
|AC_BS|Accessories|      Bike Stands|         No|
|AC_BC|Accessories|Bottles and Cages|         No|
|AC_CL|Accessories|         Cleaners|        Yes|
|AC_FE|Accessories|          Fenders|         No|
|AC_HE|Accessories|          Helmets|        Yes|
|AC_HP|Accessories|  Hydration Packs|         No|
|AC_LI|Accessories|           Lights|        Yes|
|AC_LO|Accessories|            Locks|        Yes|
|AC_PA|Accessories|         Panniers|         No|
|AC_PU|Accessories|            Pumps|        Yes|
|AC_TT|Accessories|  Tires and Tubes|        Yes|
|BI_MB|      Bikes|   Mountain Bikes|        Yes|
|BI_RB|      Bikes|       Road Bikes|        Yes|
|BI_TB|      Bikes|    Touring Bikes|        Yes|
|CL_BS|   Clothing|       Bib-Shorts|         No|
|CL_CA|   Clothing|             Caps|         No|


In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/erp/PX_CAT_G1V2_clean.csv")

25/05/26 13:17:20 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/26 13:17:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/erp/PX_CAT_G1V2_clean.csv")
df_cleaned.show()

+-----+-----------+-----------------+-----------+
|   id|        cat|           subcat|maintenance|
+-----+-----------+-----------------+-----------+
|AC_BR|Accessories|       Bike Racks|        Yes|
|AC_BS|Accessories|      Bike Stands|         No|
|AC_BC|Accessories|Bottles and Cages|         No|
|AC_CL|Accessories|         Cleaners|        Yes|
|AC_FE|Accessories|          Fenders|         No|
|AC_HE|Accessories|          Helmets|        Yes|
|AC_HP|Accessories|  Hydration Packs|         No|
|AC_LI|Accessories|           Lights|        Yes|
|AC_LO|Accessories|            Locks|        Yes|
|AC_PA|Accessories|         Panniers|         No|
|AC_PU|Accessories|            Pumps|        Yes|
|AC_TT|Accessories|  Tires and Tubes|        Yes|
|BI_MB|      Bikes|   Mountain Bikes|        Yes|
|BI_RB|      Bikes|       Road Bikes|        Yes|
|BI_TB|      Bikes|    Touring Bikes|        Yes|
|CL_BS|   Clothing|       Bib-Shorts|         No|
|CL_CA|   Clothing|             Caps|         No|


In [9]:
import duckdb

# Convert dari Spark ke Pandas
df_cleaned_PX_CAT_G1V2 = df_cleaned.toPandas()

# Connect ke DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Simpan sebagai tabel DuckDB (view)
con.register("erp_PX_CAT_G1V2_clean_view", df_cleaned_PX_CAT_G1V2)

# Simpan sebagai tabel DuckDB (bukan view)
con.execute("CREATE OR REPLACE TABLE erp_PX_CAT_G1V2_clean AS SELECT * FROM erp_PX_CAT_G1V2_clean_view")

In [7]:
con.execute("SELECT * FROM erp_PX_CAT_G1V2_clean LIMIT 10").df().style.hide(axis="index")

id,cat,subcat,maintenance
AC_BR,Accessories,Bike Racks,Yes
AC_BS,Accessories,Bike Stands,No
AC_BC,Accessories,Bottles and Cages,No
AC_CL,Accessories,Cleaners,Yes
AC_FE,Accessories,Fenders,No
AC_HE,Accessories,Helmets,Yes
AC_HP,Accessories,Hydration Packs,No
AC_LI,Accessories,Lights,Yes
AC_LO,Accessories,Locks,Yes
AC_PA,Accessories,Panniers,No


In [11]:
con.close()